In [18]:
# https://xilinx.github.io/brevitas/getting_started.html
# Brevitas model with 4-bit quantization

from torch import nn
from torch.nn import Module
import torch.nn.functional as F
import brevitas.nn as qnn
import torch 

import os
import onnx
# import torch
import numpy as np
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader, Dataset
from brevitas.nn import QuantLinear, QuantReLU
# import torch.nn as nn
from sklearn.metrics import accuracy_score
from tqdm import tqdm, trange
# import ipynbname

class QuantWeightLeNet(Module):
    def __init__(self):
        super(QuantWeightLeNet, self).__init__()
        self.conv1 = qnn.QuantConv2d(3, 6, 5, bias=True, weight_bit_width=4)
        self.relu1 = nn.ReLU()
        self.conv2 = qnn.QuantConv2d(6, 16, 5, bias=True, weight_bit_width=4)
        self.relu2 = nn.ReLU()
        self.fc1   = qnn.QuantLinear(16*5*5, 120, bias=True, weight_bit_width=4)
        self.relu3 = nn.ReLU()
        self.fc2   = qnn.QuantLinear(120, 84, bias=True, weight_bit_width=4)
        self.relu4 = nn.ReLU()
        self.fc3   = qnn.QuantLinear(84, 10, bias=True, weight_bit_width=4)

    def forward(self, x):
        out = self.relu1(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = self.relu2(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.reshape(out.shape[0], -1)
        out = self.relu3(self.fc1(out))
        out = self.relu4(self.fc2(out))
        out = self.fc3(out)
        return out

model = QuantWeightLeNet()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Target device: " + str(device))

model.to(device); # The semicolon is for not printing the model.

Target device: cpu


In [19]:
import numpy as np
import torchvision
import torchvision.transforms as transforms
import os

def quantize_image(image):
    """Quantize and binarize an image."""
    image = image.astype(np.float32)
    image = np.floor(image / (256/16))  # Example: reducing to 4-bit quantization
    return image.astype(np.float32)

def save_dataset_as_npz(data, labels, filename):
    """Save the dataset as a .npz file."""
    np.savez_compressed(filename, data=data, labels=labels)

# Load CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Quantize the images
train_images = np.array([quantize_image(image.numpy().transpose(1, 2, 0) * 255) for image, _ in train_dataset])
train_labels = np.array(train_dataset.targets)

test_images = np.array([quantize_image(image.numpy().transpose(1, 2, 0) * 255) for image, _ in test_dataset])
test_labels = np.array(test_dataset.targets)

# Save the datasets
os.makedirs('./quantized_data', exist_ok=True)
save_dataset_as_npz(train_images, train_labels, './quantized_data/cifar10_train.npz')
save_dataset_as_npz(test_images, test_labels, './quantized_data/cifar10_test.npz')

Files already downloaded and verified
Files already downloaded and verified


In [31]:
# data = np.load('./quantized_data/cifar10_train.npz')
# # print(data.files)
# print(data['data'].shape)
# print(data['labels'].shape)
# print(data['data'][328][15])
# print(data['labels'][328])

In [28]:
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import time

class CIFAR10QuantizedDataset(Dataset):
    def __init__(self, npz_file):
        data = np.load(npz_file)
        self.images = data['data']
        self.labels = data['labels']

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx].astype(np.float32) / 255.0
        label = self.labels[idx]
        image = torch.tensor(image.transpose(2, 0, 1))  # HWC to CHW format
        label = torch.tensor(label, dtype=torch.long)
        return image, label

# Load the quantized dataset
train_dataset = CIFAR10QuantizedDataset('./quantized_data/cifar10_train.npz')
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = CIFAR10QuantizedDataset('./quantized_data/cifar10_test.npz')
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Training the model
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train(model, train_loader, criterion, optimizer, num_epochs=10):
    for epoch in tqdm(range(num_epochs)):
        model.train()
        running_loss = 0.0
        for i, (images, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # running_loss += loss.item()
            # if i % 100 == 99:    # print every 100 mini-batches
            #     print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 100:.4f}')
            #     running_loss = 0.0
            running_loss += loss.item()
        # print(f'Epoch {epoch}, Loss: {running_loss / 100:.4f}')
        # running_loss = 0.0
            

    print('Finished Training')

train(model, train_loader, criterion, optimizer)


100%|██████████████████████████████████████████████████████████████████| 100/100 [11:17<00:00,  6.78s/it]

Finished Training


In [30]:
def test(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():  # Disable gradient calculation for testing
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy on the test dataset: {accuracy:.2f}%')

test(model, test_loader)

# Accuracy is almost same after 10 and 100 epochs.

Accuracy on the test dataset: 55.44%
